### Example Jupyter Code for Generation Results with Gradio Demo

```
--------------------------------------------------------
DisCo - Disentangled Control for Referring Human Dance Generation in Real World
Licensed under The Apache-2.0 license License [see LICENSE for details]
Tan Wang (TAN317@e.ntu.edu.sg)
Work done during internship at Microsoft
--------------------------------------------------------
```

Pls remember to change the path (model checkpoint, root dir, eval_save_filename, and so on) in the `manual_args`.

In [1]:
import numpy as np
print(np.__version__)

1.24.4


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_ENABLE"] = "0"

from utils.wutils_ldm import *
from agent import Agent_LDM, WarmupLinearLR, WarmupLinearConstantLR
import torch
import torch.distributed as dist
from config import BasicArgs
from utils.lib import *
from utils.args import parse_with_cf

from utils.dist import dist_init

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '23658'
os.environ["USE_LIBUV"] = "0"

# # initialize the process group
# dist.init_process_group(
#     backend="cuda:gloo",
#     rank=0,
#     world_size=1,
#     init_method=f"tcp://localhost:23456",
# )

from dataset.tsv_dataset import make_data_sampler, make_batch_data_sampler
from finetune_sdm_yaml import get_loader_info, make_data_loader
torch.multiprocessing.set_sharing_strategy('file_system')

[2024-09-26 18:01:01 <wutils_ldm.py:156> <module>] <utils.py>: Deep Learning Utils @ Chenfei Wu
/scratch/sahil/anaconda3/envs/disco1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WANDB_ENABLE: 0
2024-09-26 18:01:02,193.193 2301975:common.py:1785 setup_yaml(): python 3 env
[2024-09-26 18:01:02,424] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/scratch/sahil/anaconda3/envs/disco1/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/scratch/sahil/anaconda3/envs/disco1/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
[2024-09-26 18:01:03 <wutils.py:132> <module>] <utils.py>: Deep Learning Utils @ Chenfei Wu
[2024-09-26 18:01:03 <wutils.py:132> <module>] <utils.py>: Deep Learning Utils @ Chenfei Wu
[2024-09-26 18:01:03 <__init__.py:71> BasicArgs] Detected unknown Node katz.
[2024-09-26 18:01:03 <__init__.py:71> BasicArgs] Detected unknown Node katz.


In [3]:
from utils.args import sharedArgs

manual_args = ['--cf', 'config/ref_attn_clip_combine_controlnet/app_demo_image_edit.py', '--eval_visu', 'True', '--root_dir', '/scratch/namrata/3d_pose_gen', '--local_train_batch_size', '32', '--local_eval_batch_size', '32', '--log_dir', 'exp/tiktok_ft5', '--epochs', '20', '--deepspeed', '--eval_step', '500',
               '--save_step', '500', '--gradient_accumulate_steps', '1', '--learning_rate', '2e-4', '--fix_dist_seed', 'True', '--loss_target',
               'noise', '--unet_unfreeze_type', 'all', '--guidance_scale', '3', '--refer_sdvae', 'True', '--ref_null_caption', 'False', '--combine_clip_local', 'True', '--combine_use_mask', 'True', '--conds', 'poses','masks', '--pretrained_model', 'exp/tiktok_ft22_final/1999.pth/mp_rank_00_model_states.pt', '--eval_save_filename', 'try']
parsed_args = sharedArgs.parser.parse_args(args=manual_args)
# args = sharedArgs.parser.parse_args(args=['--cf', 'config/ref_attn_clip_combine_controlnet/app_demo_image_edit.py', '--eval_visu', 'True'])
# import pdb; pdb.set_trace()


###### process the args #######
if parsed_args.root_dir:
    BasicArgs.root_dir = parsed_args.root_dir
else:
    parsed_args.root_dir = BasicArgs.root_dir
parsed_args.pretrained_model_path = os.path.join(parsed_args.root_dir, parsed_args.pretrained_model_path)

def parse_with_cf(parsed_args):
    """This function will set args based on the input config file.
    (1) it only overwrites unset parameters,
        i.e., these parameters not set from user command line input
    (2) it also sets configs in the config file but declared in the parser
    """
    # convert to EasyDict object,
    # enabling access from attributes even for nested config
    # e.g., args.train_datasets[0].name
    args = edict(vars(parsed_args))
    if os.path.exists(parsed_args.cf):
        cf = import_filename(parsed_args.cf)
        config_args = edict(vars(cf.Args))
        override_keys = {arg[2:].split("=")[0] for arg in manual_args
                         if arg.startswith("--")}
        # import pdb;pdb.set_trace()
        for k, v in config_args.items():
            if k not in override_keys:
                setattr(args, k, v)
    else:
        raise NotImplementedError('Config filename %s does not exist.' % args.cf)
    return args

args = parse_with_cf(parsed_args)
        
args.n_gpu = T.cuda.device_count() # local size
args.local_size = args.n_gpu
if args.root_dir not in args.log_dir:
    args.log_dir = os.path.join(args.root_dir, args.log_dir)
if args.stepwise_sample_depth == -1:
    args.interpolation = None
    args.interpolate_mode = None
if args.interpolation != "interpolate":
    args.interpolate_mode = None

assert args.eval_step > 0, "eval_step must be positive"
assert args.save_step > 0, "save_step must be positive"

dist_init(args)
args.dist = args.distributed
args.nodes = args.num_nodes
args.world_size = args.num_gpus
args.train_batch_size = args.local_train_batch_size * args.world_size
args.eval_batch_size = args.local_eval_batch_size * args.world_size
#############################################

cf = import_filename(args.cf)
Net, inner_collect_fn = cf.Net, cf.inner_collect_fn

dataset_cf = import_filename(args.dataset_cf)
BaseDataset = dataset_cf.BaseDataset

# args = update_args(parsed_args, args)

# init models
logger.info('Building models...')
model = Net(args)
print(f"Args: {edict(vars(args))}")

[2024-09-26 18:01:03 <2307042429.py:73> <module>] Building models...
[2024-09-26 18:01:03 <2307042429.py:73> <module>] Building models...


distributed training ... presumbly debug with 1 GPU
Using seed 42 for rank 0
Using seed 42 for torch.cuda
Loading pre-trained image_encoder from /scratch/namrata/3d_pose_gen/diffusers/sd-image-variations-diffusers/image_encoder
Loading pre-trained vae from /scratch/namrata/3d_pose_gen/diffusers/sd-image-variations-diffusers/vae
Loading pre-trained unet from /scratch/namrata/3d_pose_gen/diffusers/sd-image-variations-diffusers/unet
use sd vae to init the controlnet condition embedding
Args: {'root_dir': '/scratch/namrata/3d_pose_gen', 'cf': 'config/ref_attn_clip_combine_controlnet/app_demo_image_edit.py', 'pretrained_model': 'exp/tiktok_ft22_final/1999.pth/mp_rank_00_model_states.pt', 'pretrained_model_lora': None, 'pretrained_model_controlnet': None, 'debug': False, 'debug_seed': False, 'debug_dataloader': False, 'log_dir': '/scratch/namrata/3d_pose_gen/exp/tiktok_ft5', 'deepspeed': True, 'use_amp': False, 'seed': 42, 'fix_dist_seed': True, 'tiktok_data_root': 'keli/dataset/TikTok_datas

In [4]:
### prepare the eval

logger.warning("Do eval_visu...")
if getattr(args, 'refer_clip_preprocess', None):
    eval_dataset = BaseDataset(args, args.val_yaml, split='val', preprocesser=model.feature_extractor)
else:
    eval_dataset = BaseDataset(args, args.val_yaml, split='val')
eval_dataloader, eval_info = make_data_loader(
    args, args.local_eval_batch_size, 
    eval_dataset)


trainer = Agent_LDM(args=args, model=model)
trainer.eval_demo_pre()

[2024-09-26 18:01:14 <287074974.py:3> <module>] Do eval_visu...
[2024-09-26 18:01:14 <287074974.py:3> <module>] Do eval_visu...


number of samples: 25
DEEPSPEED config {'train_micro_batch_size_per_gpu': 32, 'gradient_accumulation_steps': 1, 'fp16': {'enabled': True, 'loss_scale': 64}, 'flops_profiler': {'enabled': False, 'profile_step': 1, 'module_depth': -1, 'top_modules': 3, 'detailed': True}, 'tensorboard': {'enabled': True, 'output_path': '/scratch/namrata/3d_pose_gen/exp/tiktok_ft5', 'job_name': 'tensorboard_log'}}
Specify the load model path, not use deepspeed but the pytorch original load func


[2024-09-26 18:01:21 <wutils_ldm.py:462> file2data] Loaded data from /scratch/namrata/3d_pose_gen/exp/tiktok_ft22_final/1999.pth/mp_rank_00_model_states.pt
[2024-09-26 18:01:21 <wutils_ldm.py:462> file2data] Loaded data from /scratch/namrata/3d_pose_gen/exp/tiktok_ft22_final/1999.pth/mp_rank_00_model_states.pt
[2024-09-26 18:01:28 <wutils_ldm.py:707> adaptively_load_state_dict] Strictly Loaded state_dict.
[2024-09-26 18:01:28 <wutils_ldm.py:707> adaptively_load_state_dict] Strictly Loaded state_dict.
[2024-09-26 18:01:28 <agent.py:877> load_checkpoint_for_deepspeed_diff_gpu] Loaded checkpoint exp/tiktok_ft22_final/1999.pth/mp_rank_00_model_states.pt of global_step 1000
[2024-09-26 18:01:28 <agent.py:877> load_checkpoint_for_deepspeed_diff_gpu] Loaded checkpoint exp/tiktok_ft22_final/1999.pth/mp_rank_00_model_states.pt of global_step 1000


[2024-09-26 18:01:28,930] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.15.1, git-hash=unknown, git-branch=unknown
[2024-09-26 18:01:28,931] [INFO] [comm.py:652:init_distributed] cdb=None
[2024-09-26 18:01:28,932] [INFO] [config.py:733:__init__] Config mesh_device None world_size = 1
[2024-09-26 18:01:29,527] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2024-09-26 18:01:29,535] [INFO] [config.py:999:print] DeepSpeedEngine configuration:
[2024-09-26 18:01:29,536] [INFO] [config.py:1003:print]   activation_checkpointing_config  {
    "partition_activations": false, 
    "contiguous_memory_optimization": false, 
    "cpu_checkpointing": false, 
    "number_checkpoints": null, 
    "synchronize_checkpoint_boundary": false, 
    "profile": false
}
[2024-09-26 18:01:29,537] [INFO] [config.py:1003:print]   aio_config ................... {'block_size': 1048576, 'queue_depth': 8, 'thread_count': 1, 'single_submit': False, 'overlap_event

[2024-09-26 18:01:29 <agent.py:241> prepare_dist_model] Successfully built models with {'trainable': 1581790996, 'frozen': 387620071, 'trainable_fp32': 0, 'trainalbe_fp16': 1581790996, 'frozen_fp32': 303966208, 'frozen_fp16': 83653863} parameters
[2024-09-26 18:01:29 <agent.py:241> prepare_dist_model] Successfully built models with {'trainable': 1581790996, 'frozen': 387620071, 'trainable_fp32': 0, 'trainalbe_fp16': 1581790996, 'frozen_fp32': 303966208, 'frozen_fp16': 83653863} parameters


Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


In [5]:
def load_image(image):
    if not image.mode == "RGB":
        image = image.convert("RGB")

    return image



@torch.no_grad()
def inference(reference_fg, fg_mask, ref_bg_image, bg_mask, skeleton_img, *args, **kwargs):
    reference_fg = load_image(reference_fg)
    fg_mask = load_image(fg_mask)
    ref_bg_image = load_image(ref_bg_image)
    bg_mask = load_image(bg_mask)
    skeleton_img = load_image(skeleton_img)
    
    input_data = [reference_fg, fg_mask, ref_bg_image, bg_mask, skeleton_img]
    output_image = trainer.eval_demo_run(input_data, eval_dataset=eval_dataset)
    return output_image

@torch.no_grad()
def inference_masked(reference_fg, ref_bg_image, skeleton_img, *args, **kwargs):
    reference_fg = load_image(reference_fg)
    ref_bg_image = load_image(ref_bg_image)
    skeleton_img = load_image(skeleton_img)

    print(reference_fg,ref_bg_image,skeleton_img)
    
    input_data = [reference_fg, ref_bg_image, skeleton_img]
    output_image = trainer.eval_demo_run_masked(input_data, eval_dataset=eval_dataset)

    return output_image

In [6]:
import gradio as gr

title = "DisCo Demo (Video Demo Coming Soon!)"
description = """<p style='text-align: center'> <a href='https://disco-dance.github.io/' target='_blank'>Project Page</a> | <a href='https://arxiv.org/pdf/2212.11270.pdf' target='_blank'>Paper</a> | <a href='https://github.com/microsoft/X-Decoder' target='_blank'>Github Repo</a> | <a href='https://youtu.be/wYp6vmyolqE' target='_blank'>Video</a> </p>
<p>Skip the queue by duplicating this space and upgrading to GPU in settings</p>
<a href="https://huggingface.co/spaces/xdecoder/Demo?duplicate=true"><img src="https://bit.ly/3gLdBN6" alt="Duplicate Space"></a>
"""

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # DisCo Demo (Video Demo Coming Soon!)
    Start editing the human with the provided human foreground, background, and pose.
    
    Note that for self-uploaded images, TikTok-Style human images are preferred.
    
    [Project Page](https://disco-dance.github.io/) | [Github](https://github.com/Wangt-CN/DisCo)
    """)

    with gr.Row():
        with gr.Column(min_width=400, scale=2):
            input_fg = gr.Image(type='pil', label="Foreground Image")
            gr.Examples(examples=["./demo_data/fg/masked_images/00035.png", "./demo_data/fg/masked_images/00335.png", "./demo_data/fg/masked_images/00147.png", "./demo_data/fg/masked_images/00072.png", "./demo_data/fg/masked_images/00115.png"], inputs=input_fg)

            input_bg = gr.Image(type='pil', label="Background Image")
            gr.Examples(examples=["./demo_data/bg/masked_images/00035.png", "./demo_data/bg/masked_images/00335.png", "./demo_data/bg/masked_images/00147.png", "./demo_data/bg/masked_images/00072.png", "./demo_data/bg/masked_images/00115.png"], inputs=input_bg)

            input_pose = gr.Image(type='pil', label="Target Pose", scale=1)
            gr.Examples(examples=["./demo_data/pose_img/0049.png", "./demo_data/pose_img/0198.png", "./demo_data/pose_img/0213.png", "./demo_data/pose_img/0264.png", "./demo_data/pose_img/0144.png", "./demo_data/pose_img/0054.png"], inputs=input_pose)

            btn = gr.Button("Generate")
        
        with gr.Column(min_width=150):
            output_img = gr.Image(type='pil', label="Edited Human Image")

    btn.click(inference_masked, inputs=[input_fg, input_bg, input_pose], outputs=[output_img], concurrency_limit=2)
    
demo.launch(share=True, max_threads=4)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://36cfad975c622f4ac0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C54F610> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E94ACE750> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E94ACE610>
Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


100%|██████████| 50/50 [00:03<00:00, 13.41it/s]


<PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C689610> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C689D10> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C6C2910>
Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


100%|██████████| 50/50 [00:04<00:00, 11.45it/s]


...

100%|██████████| 50/50 [00:02<00:00, 17.72it/s]


<PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C6B4C10> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C6B5350> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C6C0710>
Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


100%|██████████| 50/50 [00:03<00:00, 14.47it/s]


<PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C54F550> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C6C2D50> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C6C2450>
Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


100%|██████████| 50/50 [00:03<00:00, 14.76it/s]


In [8]:
import torch
import numpy as np
from scipy.linalg import sqrtm
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image

# Step 1: Load the model
model = torch.load('exp/tiktok_ft22_final/1999.pth/mp_rank_00_model_states.pt')
model.eval()  # Set the model to evaluation mode

# Step 2: Generate images (Assume model outputs images directly)
def generate_images(model, num_images, device='cuda'):
    model.to(device)
    images = []
    for _ in range(num_images):
        # Assuming your model generates an image with a forward pass
        with torch.no_grad():
            img = model()  # Replace with actual model generation code
            images.append(img.cpu())
    return torch.stack(images)

# Step 3: Use Inception to extract features
def get_inception_features(images, batch_size=16, device='cuda'):
    inception = models.inception_v3(pretrained=True, transform_input=False).to(device)
    inception.eval()
    
    # Define the transform for input to Inception
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Process images in batches
    loader = DataLoader(images, batch_size=batch_size, shuffle=False)
    features = []
    
    with torch.no_grad():
        for img_batch in loader:
            img_batch = img_batch.to(device)
            feat = inception(img_batch).cpu().numpy()
            features.append(feat)
    
    return np.concatenate(features)

# Step 4: Calculate the FID Score
def calculate_fid(mu1, sigma1, mu2, sigma2):
    # Compute FID using mean and covariance
    diff = mu1 - mu2
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    
    # Handle imaginary numbers from sqrtm
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

# Step 5: Main function to calculate FID
def fid_score(model, real_images, num_generated_images=1000, device='cuda'):
    # Generate images from the model
    generated_images = generate_images(model, num_generated_images, device)

    # Extract Inception features for real and generated images
    real_features = get_inception_features(real_images, device=device)
    generated_features = get_inception_features(generated_images, device=device)

    # Calculate mean and covariance of the features
    mu_real, sigma_real = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu_gen, sigma_gen = np.mean(generated_features, axis=0), np.cov(generated_features, rowvar=False)

    # Calculate FID score
    fid = calculate_fid(mu_real, sigma_real, mu_gen, sigma_gen)
    return fid

# Example usage
# Load real images dataset (this is an example, you should provide real images)
real_images_dataset = ImageFolder(root='demo_data/fg/images', transform=transforms.ToTensor())
real_images = DataLoader(real_images_dataset, batch_size=32)

# Calculate FID score
fid_value = fid_score(model, real_images)
print(f'FID Score: {fid_value}')


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.95 GiB. GPU 0 has a total capacity of 79.15 GiB of which 2.27 GiB is free. Process 2294293 has 54.54 GiB memory in use. Process 2301360 has 2.61 GiB memory in use. Process 2314023 has 10.37 GiB memory in use. Including non-PyTorch memory, this process has 9.30 GiB memory in use. Of the allocated memory 8.40 GiB is allocated by PyTorch, and 83.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

<PIL.Image.Image image mode=RGB size=256x256 at 0x7F3048797C90> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C489B90> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2E8C48A790>
Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


100%|██████████| 50/50 [00:03<00:00, 13.33it/s]


<PIL.Image.Image image mode=RGB size=256x256 at 0x7F30486C4C10> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F304847F7D0> <PIL.Image.Image image mode=RGB size=256x256 at 0x7F304847E110>
Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


100%|██████████| 50/50 [00:03<00:00, 13.31it/s]


: 